In [1]:
def load_csv(filename):
    import pandas as pd
    df = pd.read_csv(filename,sep=",")
    df.rename(columns=lambda x: x.replace(" ", "_"),inplace=True)
    return df

In [2]:
class LogisticRegression:   
    def __init__(self, Predictors, Response, learning_rate=0.001, iterations=10000):
        import numpy as np
        """
        Initialize Parameters
        """
        # Predictors are the input variables from the data
        self.Predictors = np.concatenate((np.ones((np.shape(Predictors)[0],1),dtype=int),Predictors),axis=1)
        
        # Response is the output variable from the data
        self.Response = Response          
        self.lr = learning_rate
        self.iters = iterations
        
        # There is a weight associated with the intercept and a weight for each feature
        self.m,self.n = self.Predictors.shape
        self.weights = np.ones(self.n,dtype=int)
        self.weights[0] = 0
        
    def sigm(self,Z):
        """
        Sigmoid Function 
        sigm(Z) = exp(Z) / (1 + exp(Z)) = 1 / (1 + exp(-Z) 
        """
        import numpy as np
        return 1 / (1 + np.exp(-Z))
    
    def mu(self,X):
        """
        mu(X) = E[y | x] = p(y=1 | x) = sigm(w.T * X)
        """
        import numpy as np
        return self.sigm(np.dot(X,self.weights.T))
    
    def classification(self, ridge):
        import numpy as np
        """
        Ridge Regression is as simple as adding the L2 regularization to the gradient
        g'(w) = g(w) + lambda*w
        Weights = Weights - Learning_Rate * Gradient
        """
        diff = np.inf
        iterations = self.iters
        
        while diff > 0.01 and iterations > 0:  
            p_hat = self.mu(self.Predictors)
            gradient = np.dot(self.Predictors.T,(p_hat - self.Response)) + ridge*self.weights
            self.weights = self.weights - np.dot(self.lr,gradient)
                        
            diff = np.abs(gradient).sum()
            iterations -= 1
        return self.weights
    
    def decision_rule(self,Y):
        """
        y_hat(x) = 1 if and only if p(y=1 | x) >= 0.5
        """
        for i in range(len(Y)):
            if Y[i] >= 0.5:
                Y[i] = 1
            else:
                Y[i] = 0
        return Y
            
    def prediction(self, Predictors = []):
        import numpy as np
        if not Predictors: Predictors = self.Predictors
        y_pred = self.mu(Predictors)
        y_pred = self.decision_rule(y_pred)
        return y_pred

In [3]:
data = load_csv('Breast_Cancer_Wisconsin.csv')
data.columns

Index(['ID_Number', 'Clump_Thickness', 'Unif_Cell_Size', 'Unif_Cell_Shape',
       'Marg_Adhesion', 'Single_Epith_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class'],
      dtype='object')

In [4]:
for i in data.columns:
    print(data[i].value_counts())

1182404    6
1276091    5
1198641    3
897471     2
1116192    2
          ..
1232225    1
1236043    1
1241232    1
1241559    1
814265     1
Name: ID_Number, Length: 645, dtype: int64
1     145
5     130
3     108
4      80
10     69
2      50
8      46
6      34
7      23
9      14
Name: Clump_Thickness, dtype: int64
1     384
10     67
3      52
2      45
4      40
5      30
8      29
6      27
7      19
9       6
Name: Unif_Cell_Size, dtype: int64
1     353
2      59
10     58
3      56
4      44
5      34
6      30
7      30
8      28
9       7
Name: Unif_Cell_Shape, dtype: int64
1     407
3      58
2      58
10     55
4      33
8      25
5      23
6      22
7      13
9       5
Name: Marg_Adhesion, dtype: int64
2     386
3      72
4      48
1      47
6      41
5      39
10     31
8      21
7      12
9       2
Name: Single_Epith_Cell_Size, dtype: int64
1     402
10    132
2      30
5      30
3      28
8      21
4      19
?      16
9       9
7       8
6       4
Name: Bare_Nuclei, d

In [5]:
data[:][22:25]

,ID_Number,Clump_Thickness,Unif_Cell_Size,Unif_Cell_Shape,Marg_Adhesion,Single_Epith_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
22,1056784,3,1,1,1,2,1,2,1,1,2
23,1057013,8,4,5,1,2,?,7,3,1,4
24,1059552,1,1,1,1,2,1,3,1,1,2


In [6]:
import numpy as np
data.replace({'?': np.nan}, inplace=True)
data.fillna(int(data['Bare_Nuclei'].median()),inplace=True)
data[:][22:25]

,ID_Number,Clump_Thickness,Unif_Cell_Size,Unif_Cell_Shape,Marg_Adhesion,Single_Epith_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
22,1056784,3,1,1,1,2,1,2,1,1,2
23,1057013,8,4,5,1,2,1,7,3,1,4
24,1059552,1,1,1,1,2,1,3,1,1,2


In [7]:
import pandas as pd
drop_list = ['ID_Number','Class']
Predictors = data.drop(drop_list,axis=1)
Predictors['Bare_Nuclei'] = pd.to_numeric(Predictors['Bare_Nuclei'])
Response = data['Class'].replace({2:0,4:1})
print(Response.value_counts())

0    458
1    241
Name: Class, dtype: int64


In [8]:
x = [[1,5,1,1,1,1,2,3,1,1],[1,7,3,2,10,5,2,5,4,4],[1,10,5,5,3,6,1,2,1,1]]
regressor = LogisticRegression(Predictors, Response)
for i in range(16):
    ridge = i / 10
    regressor.classification(ridge)
    pred = regressor.prediction(x)
    print('lambda = ',ridge)
    print('Weights: ',regressor.weights)
    print("Predictions")
    print(pred)
    print(); print()

lambda =  0.0
Weights:  [-9.70598476  0.53401165  0.01158703  0.3236618   0.23733171  0.05808009
  0.42801016  0.41164728  0.15819723  0.53487162]
Predictions
[0. 1. 1.]


lambda =  0.1
Weights:  [-8.93606987  0.48936897  0.06248099  0.32755298  0.21859973  0.04222172
  0.43185031  0.37752581  0.16854085  0.45442122]
Predictions
[0. 1. 1.]


lambda =  0.2
Weights:  [-8.57172152  0.47008184  0.11648186  0.33790397  0.21035318  0.02939497
  0.45110103  0.36236427  0.18552719  0.41640793]
Predictions
[0. 1. 1.]


lambda =  0.3
Weights:  [-8.29611585  0.45218174  0.15939556  0.34454998  0.20355064  0.01692078
  0.46628613  0.34779211  0.1992056   0.38939407]
Predictions
[0. 1. 1.]


lambda =  0.4
Weights:  [-8.07536376e+00  4.36540424e-01  1.95993840e-01  3.50042231e-01
  1.98601150e-01  5.60849878e-03  4.80010204e-01  3.34630945e-01
  2.11754178e-01  3.69225965e-01]
Predictions
[0. 1. 1.]


lambda =  0.5
Weights:  [-7.89173492e+00  4.22701760e-01  2.28249621e-01  3.54966028e-01
  1.951187